For Mask-RCNN
用來重新排序及分trainingdata , testingdata

## Define a data converter for data collected by Unity

In [1]:
import os
import cv2
import json
import random
import shutil
from PIL import Image
from glob import glob
import annotated_images
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
class UnityDataConverter():
  def __init__(self, data_dir):
    self.data_dir = data_dir
    
  def delete_outlier(self):
    '''
    delete first image and put all into same file
    '''
    scenes = sorted(os.listdir(self.data_dir))
    
    for scene in scenes:      
        target_jpg = self.data_dir+'/'+ scene +'/'+'1.main.jpg'
        target_png = self.data_dir+'/'+ scene +'/'+'1.main.seg.png'
        target_json = self.data_dir+'/'+ scene +'/'+'1.main.json'
        target_depth = self.data_dir+'/'+ scene +'/'+'1.main.depth.png'
        src = self.data_dir+'/'+ scene
        try:
            os.remove(target_jpg)
            print(f'\'delete\':{target_jpg}')                  
        except:
            print(f'{target_jpg} not found')
                
        try:
            os.remove(target_png)
            print(f'\'delete\':{target_png}')                  
        except:
            print(f'{target_png} not found')
            
        try:
            os.remove(target_json)
            print(f'\'delete\':{target_json}')                  
        except:
            print(f'{target_json} not found')
            
        try:
            os.remove(target_depth)
            print(f'\'delete\':{target_depth}')                  
        except:
            print(f'{target_depth} not found')
        
        images_path_jpg = sorted(glob(os.path.join(self.data_dir+'/'+scene+'/*.main.jpg')))
        images_path_png = sorted(glob(self.data_dir+'/'+scene+'/*.main.seg.png'))
        json_path = sorted(glob(self.data_dir+'/'+scene+'/*.main.json'))     
        depth_path = sorted(glob(self.data_dir+'/'+scene+'/*.main.depth.png'))
        
        print(len(images_path_jpg))
        
        for i in range (len(images_path_jpg)):
            
            folder_name = self.data_dir+'/'+scene+ '/' + scene + '_'+ str(i) 
            src_jpg = os.path.join(folder_name +'.jpg')
            src_png = os.path.join(folder_name +'.main.seg.png')
            src_json = os.path.join(folder_name +'.main.json')
            src_depth = os.path.join(folder_name +'.main.depth.png')
            
#           print(f'image:{images_path_jpg[i]} /src_jpg:{src_jpg}')
            os.rename(images_path_jpg[i],src_jpg)
            os.rename(images_path_png[i],src_png)
            os.rename(json_path[i],src_json)
            os.rename(depth_path[i],src_depth)
        
        dst = './all_image/'
        if not os.path.exists('./all_image'):
            os.mkdir('./all_image')
            
            
        shutil.copytree(src, dst,dirs_exist_ok=True)
              
  def split_train_test_val(self, input_dir = './all_image', dst = './all_image/split_dir'):
    train_dst = dst+'/train'
    test_dst = dst+'/val'
    
    if not os.path.exists(dst):
        os.mkdir(dst)
        os.mkdir(train_dst)
        os.mkdir(test_dst)
                           
    all_file = glob(os.path.join( input_dir + '/*.jpg'))
    #print(all_file)
    
    training_data, testing_data = train_test_split(all_file, test_size=0.2, random_state=0)
    
    for i in range (len(training_data)):
        shutil.move(training_data[i], train_dst)
    for i in range (len(testing_data)):
        shutil.move(testing_data[i], test_dst)
                           
    train_num = len(training_data)
    val_num = len(testing_data)
    tatol_num = train_num + val_num 

    print("Split successfully")
    print(f'\'train\': {train_num},\'val\': {val_num}, \'total\': {tatol_num}')
    
  def process_seg_img(self,rootdir = './all_image/split_dir/val'):
     
    #get the file name in split folder 
    #rootdir = './all_image/split_dir/val'
    #rootdir = './all_image/split_dir/train'
    
    width = 640
    height = 480
    seg_file = rootdir +'.json'  # store info
    json_path = []
    seg_path = []
    image_list = []
    annotation_list = []
    file_name_path = []
    null= None
    
    for rootdir, dirs, files in os.walk(rootdir):
        print('done') 
        #print(files) #files: image
        
    for i in range(len(files)):
        basename = os.path.basename(files[i])
        file_name = os.path.splitext(basename)[0]
        file_name_path.append(file_name)
        json_path.append('./all_image/'+ file_name + '.main.json')
        seg_path.append('./all_image/'+ file_name+ '.main.seg.png')
      
    num_annos = 0  # Number of annotations
    num_imgs = 0 # Number of images
    
    for i in range (len(json_path)):
        
    # Segementation   
         ###read json   
        with open(json_path[i], 'r') as load_f:
            dataset= []
            load_dict = json.load(load_f)
            objs = load_dict['objects']
            for obj in objs:
                name = obj['class']
                bbox = obj['bounding_box']
                xmin = bbox['top_left'][0]
                ymin = bbox['top_left'][1]
                xmax = bbox['bottom_right'][0]
                ymax = bbox['bottom_right'][1]
                # print(xmin, ymin, xmax, ymax)
                if xmin < 0: xmin = 0
                if xmax < 0: xmax = 0
                if ymin < 0: ymin = 0
                if ymax < 0: ymax = 0
                if xmin > width: xmin = 640 #width
                if xmax > width: xmax = 640 #width
                if ymin > height: ymin = 480 #height
                if ymax > height: ymax = 480 #height
                x_lenth = xmax-xmin
                y_lenth = ymax-ymin
                dataset.append(xmin)
                dataset.append(ymin)
                dataset.append(x_lenth)
                dataset.append(y_lenth)
                
                if name == 'WAM_V':
                    category = 1
                elif name == 'Boat':
                    category = 2
                elif name == 'BigBoat':
                    category = 3
                elif name == null :
                    category = 0
            ###      
            img = cv2.imread(seg_path[i])
            array = np.array(img) 
            array_new = array  
            cnt=0
            for j in range (480):
                for k in range (640):
                    for l in range (3) :
                        if array_new[j][k][l] !=0 :
                            array_new[j][k][l] = 255
                            cnt+=1
                        else : 
                            pass      
            image_order = (re.findall(r"\d+",seg_path[i]))[1]
            #cv2.imwrite(self.data_dir+'/'+scene+'/'+image_order+'.new_main.seg.png', array_new)
            #mask_img = cv2.imread(self.data_dir+'/'+scene+'/'+ image_order +'.new_main.seg.png') 
            
            cv2.imwrite('./all_image/'+ file_name_path[i] +'.new_main.seg.png', array_new)
            mask_img = cv2.imread('./all_image/'+ file_name_path[i] +'.new_main.seg.png') 
            
            #cv2_imshow(mask_img)     
            gray = cv2.cvtColor(mask_img,cv2.COLOR_BGR2GRAY)  
            ret, binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)  
            contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
            new_contours = []
            
            for j in range (len(contours)):
                new_contours.extend(contours[j].flatten().tolist())
        
            image_dict ={"width": 640 ,
                              "date_captured":null , 
                              "license": 0, 
                              "url": null , 
                              "file_name":files[i],
                               "id": i , 
                               "height": 480}
            annotation_dict = {"image_id": i,
                                "segmentation":[new_contours], 
                                "bbox": dataset,
                                "area": cnt,
                                "iscrowd": 0,
                                "category_id" : category,
                                "id":i }
                
            image_list.append(image_dict)
            annotation_list.append(annotation_dict)
            
    with open(seg_file, 'w') as f: 
        final_dic = {"info": {"contributor": null, "version": null, "description": null, 
                     "year": 2023, "url": null, "date_created": "2023-03-18 00:52:14.378127"},
                     "images": image_list,
                     "licenses": [{"url": null, "name": null, "id": 0}], 
                     "type": "instances", 
                     "annotations": annotation_list,
                      "categories": [{"supercategory": null, "name": "_background_", "id": 0}, 
                                     {"supercategory": null, "name": "WAM_V", "id": 1},
                                     {"supercategory": null, "name": "Boat", "id": 2},
                                     {"supercategory": null, "name": "BigBoat", "id": 3}
                                    ]
                    }
        
        json.dump(final_dic, f) 
    print('finished')
        

## Delete outlier

In [3]:
data_dir = './wamv_boat_bigboat'
converter = UnityDataConverter(data_dir)
converter.delete_outlier()

'delete':./wamv_boat_bigboat/Scene1/1.main.jpg
'delete':./wamv_boat_bigboat/Scene1/1.main.seg.png
'delete':./wamv_boat_bigboat/Scene1/1.main.json
'delete':./wamv_boat_bigboat/Scene1/1.main.depth.png
200
'delete':./wamv_boat_bigboat/Scene2/1.main.jpg
'delete':./wamv_boat_bigboat/Scene2/1.main.seg.png
'delete':./wamv_boat_bigboat/Scene2/1.main.json
'delete':./wamv_boat_bigboat/Scene2/1.main.depth.png
200


## Split training/testing dataset

In [4]:
converter.split_train_test_val(input_dir = './all_image', dst = './all_image/split_dir')

Split successfully
'train': 320,'val': 80, 'total': 400


## Create converter

In [5]:
data_dir = './wamv_boat_bigboat'
converter = UnityDataConverter(data_dir)
rootdir = './all_image/split_dir/val'
converter.process_seg_img(rootdir)

done
finished


In [ ]:
rootdir = './all_image/split_dir/train'
converter.process_seg_img(rootdir)